# 0.0 Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import random

import matplotlib.pyplot as plt
import seaborn as sns

import scikitplot as skplt
from sklearn.model_selection import StratifiedShuffleSplit as sss

from imblearn.ensemble import RUSBoostClassifier

## 0.1 Load data

In [2]:
home_path = "/home/marcos/Documentos/comunidade_DS/pa004_health_insurance_cross_sell/"

In [3]:
df_tree_train = pd.read_pickle(home_path + "interim/df6_bal_tree_train.pkl")

df_tree_validation = pd.read_pickle(home_path + "interim/df6_bal_tree_val.pkl")

## 0.2 Separate train and validation data

In [4]:
df7_train = df_tree_train.copy()

df7_validation = df_tree_validation.copy()

In [5]:
df7_train.isna().sum()

vintage                 0
annual_premium          0
age                     0
region_code             0
policy_sales_channel    0
vehicle_hist            0
vehicle_damage          0
previously_insured      0
weight_ages             0
id                      0
response                0
dtype: int64

In [6]:
df7_validation.isna().sum()

vintage                 0
annual_premium          0
age                     0
region_code             0
policy_sales_channel    3
vehicle_hist            0
vehicle_damage          0
previously_insured      0
weight_ages             0
id                      0
response                0
dtype: int64

In [7]:
df7_validation.dropna(axis=0, inplace=True)
df7_validation.isna().sum()

vintage                 0
annual_premium          0
age                     0
region_code             0
policy_sales_channel    0
vehicle_hist            0
vehicle_damage          0
previously_insured      0
weight_ages             0
id                      0
response                0
dtype: int64

## 0.3 Helper Functions

In [8]:
# precision_at_k
def precision_at_k(data, k):
    data = data.reset_index(drop=True)
    data['n_samples'] = data.index + 1
    data['precision_at_k'] = data['response'].cumsum() / data['n_samples']
    return data.loc[k, 'precision_at_k']

# recall_at_k
def recall_at_k(data, k):
    data = data.reset_index(drop=True)
    data['recall_at_k'] = data['response'].cumsum() / data['response'].sum()
    return data.loc[k, 'recall_at_k']

# model predict
def model_evaluate(model, model_name, data_train, data_val, k):
    # separate X and Y data:
    xtrain = data_train.drop(['id', 'response'], axis=1)
    ytrain = data_train.response
    xval = data_val.drop(['id', 'response'], axis=1)
    yval = data_val.response
    
    # fit and predict_proba:
    model.fit(xtrain, ytrain)
    yhat_proba = model.predict_proba(xval)
    
    # transform yhat_proba to 1D-array
    yhat_proba_1d = yhat_proba[:, 1].tolist()
    
    # include in dataframe
    validation_data = data_val.copy()
    validation_data['score'] = yhat_proba_1d
    # sort
    validation_data = validation_data.sort_values('score', ascending=False)
    
    # plot
    skplt.metrics.plot_cumulative_gain(yval, yhat_proba);
    
    return pd.DataFrame({'Model name':model_name,
                         'precision_at_k':precision_at_k(validation_data, k),
                         'recall_at_k':recall_at_k(validation_data, k)}, index=[0])

# model fit
def model_fit(model, data):
    # separate X and Y data:
    xtrain = data.drop(['id', 'response'], axis=1)
    ytrain = data.response
    
    # fit
    model_fitted = model.fit(xtrain, ytrain)
    
    return model_fitted



def cross_validation(model, model_name, training_data, k_top, kfolds, verbose=False):
    # X separate X and Y data:
    xtraining = training_data.drop(['response'], axis=1)
    ytraining = training_data.response
    
    # cross-validation:
    cv = sss(n_splits=kfolds)
    prec_k_list = []
    rec_k_list = []
    for train_index, prim_val_index in cv.split(xtraining, ytraining):
        X_training, X_prim_val = xtraining.iloc[train_index], xtraining.iloc[prim_val_index]
        y_training, y_prim_val = ytraining.iloc[train_index], ytraining.iloc[prim_val_index]
        
        # remove id from training, and create new validation without id
        X_training = X_training.drop(['id'], axis=1)
        X_prim_val_no_id = X_prim_val.drop(['id'], axis=1)
        
        # fit and predict_proba
        model.fit(X_training, y_training)
        yhat_proba = model.predict_proba(X_prim_val_no_id)
        
        # transform yhat_proba to 1D-array
        yhat_proba_1d = yhat_proba[:, 1].tolist()
        
        # reconstruct dataframe
        prim_val = pd.concat([X_prim_val, y_prim_val], axis=1)
        prim_val['score'] = yhat_proba_1d
        prim_val = prim_val.sort_values('score', ascending=False)
        
        # evaluate accuracy and store in list
        prec_k_list.append(precision_at_k(prim_val, k_top))
        rec_k_list.append(recall_at_k(prim_val, k_top))
    
    #evaluate mean and std
    prec_k_pred = np.round(np.mean(prec_k_list), 4).astype(str) + '+/-' + np.round(np.std(prec_k_list), 4).astype(str)
    rec_k_pred = np.round(np.mean(rec_k_list), 4).astype(str) + '+/-' + np.round(np.std(rec_k_list), 4).astype(str)
    
    return pd.DataFrame({'Model name':model_name,
                         'precision_at_k':prec_k_pred,
                         'recall_at_k':rec_k_pred}, index=[0])

# 8.0 Hyperparameter Fine Tunning

## 8.1 RUS Boost Classifier

In [9]:
param = {'n_estimators':[2000, 2500, 3000, 3500],
         'eta':[0.01, 0.03],
         'gamma':[3, 10, 20],
         'max_depth':[3, 5, 9],
         'subsample':[0.1, 0.5, 0.7],
         'colsample_bytree':[0.3, 0.7, 0.9],
         'min_child_weight':[3, 8, 15],
        }

MAX_EVAL = 5

In [10]:
final_result = pd.DataFrame()
hyperparam_lst = []

for i in range(MAX_EVAL):
    #randomly selected hyperparameters
    hp = {k:random.sample(v, 1)[0] for k, v in param.items()}
    #print the selected ones
    print(hp)
    #store in list
    hyperparam_lst.append(hp.copy())
    #model
    model = RUSBoostClassifier(n_estimators=150, sampling_strategy=0.15, replacement=True, random_state=50)
    result = cross_validation(model, 'RUS Boost Classifier', df7_train, 20000, 5)
    iter_nr = pd.Series([i], name='Iter_nr')
    result_w_nr = pd.concat([result, iter_nr], axis=1)
    final_result = pd.concat([final_result, result_w_nr], ignore_index=True)

final_result.sort_values(by=['precision_at_k'], ascending=False, inplace=True, ignore_index=True)
final_result

{'n_estimators': 3500, 'eta': 0.03, 'gamma': 20, 'max_depth': 5, 'subsample': 0.1, 'colsample_bytree': 0.9, 'min_child_weight': 15}


KeyboardInterrupt: 

In [ ]:
final_result.to_pickle(home_path + 'interim/rus_cv.pkl')

In [ ]:
# select the hyperparameters that resulted in best accuracy
best_param_nr = final_result.loc[0, 'Iter_nr']

param_tuned = hyperparam_lst[best_param_nr]

In [ ]:
#model
model_tuned = RUSBoostClassifier(objective='binary:logistic',
                                    use_label_encoder=False,
                                    n_estimators=param_tuned['n_estimators'],
                                    eta=param_tuned['eta'],
                                    gamma=param_tuned['gamma'],
                                    max_depth=param_tuned['max_depth'],
                                    n_jobs=-1,
                                    subsample=param_tuned['subsample'],
                                    colsample_bytree=param_tuned['colsample_bytree'],
                                    min_child_weight=param_tuned['min_child_weight'],
                                    verbosity=0)

In [ ]:
model_tuned_ranking = model_evaluate(model_tuned, "RUS Boost Classifier", df7_train, df7_validation, 20000)

model_tuned_ranking

In [ ]:
rus_tuned = model_fit(model_tuned, df7_train)

In [ ]:
pickle.dump(rus_tuned, open(home_path + "ml_models_comp/rus_tuned.pkl", "wb"))

### 8.1.1 Validation

In [ ]:
validation_result = pd.DataFrame()

for i in range(final_result.shape[0]):
    j = final_result.loc[i, 'Iter_nr']
    param_dict = hyperparam_lst[j]
    # print results
    print(param_dict)
    
    #model
    model = RUSBoostClassifier(n_estimators=150, sampling_strategy=0.15, replacement=True, random_state=50)
    result = model_evaluate(model, 'RUS Boost Classifier', df7_train, df7_validation, 20000)
    iter_nr = pd.Series([j], name='Iter_nr')
    result_w_nr = pd.concat([result, iter_nr], axis=1)
    validation_result = pd.concat([validation_result, result_w_nr], ignore_index=True)

validation_result.sort_values(by=['recall_at_k'], ascending=False, inplace=True, ignore_index=True)
validation_result

In [ ]:
# select the hyperparameters that resulted in best accuracy
best_param_nr = validation_result.loc[0, 'Iter_nr']

param_validated = hyperparam_lst[best_param_nr]

In [ ]:
# model
model = RUSBoostClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          n_estimators=param_validated['n_estimators'],
                          eta=param_validated['eta'],
                          gamma=param_validated['gamma'],
                          max_depth=param_validated['max_depth'],
                          n_jobs=-1,
                          subsample=param_validated['subsample'],
                          colsample_bytree=param_validated['colsample_bytree'],
                          min_child_weight=param_validated['min_child_weight'],
                          verbosity=0)

rus_rank = model_evaluate(model, "RUS Boost Classifier", df7_train, df7_validation, 20000)
rus_rank

In [ ]:
rus_validated = model_fit(model, df7_train)

In [ ]:
pickle.dump(rus_validated, open(home_path + "ml_models_comp/rus_validated.pkl", "wb"))